# OpenAI Function Calling

In [32]:
import os
import json
from openai import OpenAI
from IPython.display import display, JSON

OPENAI_API_KEY_PATH = '/home/loc/Documents/OPENAI_API_KEY.txt'
llm_model = 'gpt-3.5-turbo-0613'

with open(OPENAI_API_KEY_PATH) as f:
    OPENAI_API_KEY = f.read().strip()
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

client = OpenAI(api_key = OPENAI_API_KEY)

In [33]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API

def get_current_weather(location, unit='fahrenheit'):
    '''
    Get the current weather in a given location
    '''
    weather_info = {
        'location': location,
        'temperature': '72',
        'unit': unit,
        'forecast':['sunny','windy']
    }
    return json.dumps(weather_info) # convert to string

In [34]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [35]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [36]:
# Get response
response = client.chat.completions.create(
        model = llm_model,
        messages = messages,
        temperature = 0
)

# Print out
display(JSON(response.json()))

/home/loc/miniconda3/envs/venv/lib/python3.8/site-packages/IPython/core/display.py:618: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>

In [37]:
# Get response
response = client.chat.completions.create(
    model = llm_model,
    messages = messages,
    functions = functions,
    temperature = 0)

# Print out
display(JSON(response.json()))

<IPython.core.display.JSON object>

In [55]:
response_message = response.choices[0].message

In [56]:
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)

In [58]:
response_message.content

In [61]:
response_message.function_call

FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather')

In [62]:
response_message.function_call.arguments

'{\n  "location": "Boston, MA"\n}'

In [64]:
args = json.loads(response_message.function_call.arguments)
args

{'location': 'Boston, MA'}

In [65]:
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [67]:
messages = [
    {
        'role':'user',
        'content':'hi!'
    }
]

In [68]:
# Get response
response = client.chat.completions.create(
    model = llm_model,
    messages = messages,
    functions = functions
)

# Print out
display(JSON(response.json()))

<IPython.core.display.JSON object>

In [69]:
# Get response
response = client.chat.completions.create(
    model = llm_model,
    messages = messages,
    functions = functions,
    function_call = 'auto',
)

# Print out
display(JSON(response.json()))

<IPython.core.display.JSON object>

In [71]:
# Get response
response = client.chat.completions.create(
    model = llm_model,
    messages = messages,
    functions = functions,
    function_call = 'none',
)

# Print out
display(JSON(response.json()))

<IPython.core.display.JSON object>

In [72]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]

# Get response
response = client.chat.completions.create(
    model = llm_model,
    messages = messages,
    functions = functions,
    function_call = 'none',
)

# Print out
display(JSON(response.json()))

<IPython.core.display.JSON object>

In [73]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]

# Get response
response = client.chat.completions.create(
    model = llm_model,
    messages = messages,
    functions = functions,
    function_call = 'auto',
)

# Print out
display(JSON(response.json()))

<IPython.core.display.JSON object>

In [74]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

# Get response
response = client.chat.completions.create(
    model = llm_model,
    messages = messages,
    functions = functions,
    function_call = {"name": "get_current_weather"}, # call specific function forcely
)

# Print out
display(JSON(response.json()))

<IPython.core.display.JSON object>

In [76]:
messages = [
    {
        "role": "user",
        "content": "What's the weather and the celsius temperature in Boston!",
    }
]

# Get response
response = client.chat.completions.create(
    model = llm_model,
    messages = messages,
    functions = functions,
    function_call = {"name": "get_current_weather"}, # call specific function
)

# Print out
display(JSON(response.json()))

<IPython.core.display.JSON object>

In [79]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

# Get response
response = client.chat.completions.create(
    model = llm_model,
    messages = messages,
    functions = functions,
    function_call = {"name": "get_current_weather"}, # call specific function
)

# Print out
display(JSON(response.json()))

<IPython.core.display.JSON object>

In [80]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]

# Get response
response = client.chat.completions.create(
    model = llm_model,
    messages = messages,
    functions = functions,
    function_call = {"name": "get_current_weather"}, # call specific function
)

# Print out
display(JSON(response.json()))

<IPython.core.display.JSON object>

In [82]:
args = json.loads(response.choices[0].message.function_call.arguments)
args

{'location': 'Boston, MA'}

In [83]:
observation = get_current_weather(args)
observation

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [84]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)
messages

[{'role': 'user', 'content': "What's the weather like in Boston!"},
 {'role': 'function',
  'name': 'get_current_weather',
  'content': '{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'}]

In [85]:
# Get response
response = client.chat.completions.create(
    model = llm_model,
    messages = messages,
)

# Print out
display(JSON(response.json()))

/home/loc/miniconda3/envs/venv/lib/python3.8/site-packages/IPython/core/display.py:618: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>